## Ranking Matrix Factorization Recommender model on both implicit and explicit data

---
Library Importing

In [1]:
import numpy as np 
import pandas as pd 

from recommenders.datasets import movielens

import turicreate

---
Data Loading & prep.

In [2]:
data = movielens.load_pandas_df(
    size='100k',
    header=['UserId', 'MovieId', 'Rating', 'Timestamp'],
    title_col='Title'
)

data.loc[:, 'Rating'] = data['Rating'].astype(np.float32)
data.head()

100%|██████████| 4.81k/4.81k [00:10<00:00, 473KB/s]  


,UserId,MovieId,Rating,Timestamp,Title
0,196,242,3.0,881250949,Kolya (1996)
1,63,242,3.0,875747190,Kolya (1996)
2,226,242,5.0,883888671,Kolya (1996)
3,154,242,3.0,879138235,Kolya (1996)
4,306,242,5.0,876503793,Kolya (1996)


In [13]:
movies = pd.Series(data['Title'].values, index=data['MovieId'].values).drop_duplicates()
movie_arr = np.array(movies.values)

In [3]:
sf = turicreate.SFrame({
    'user': data['UserId'].values,
    'movie': data['MovieId'].values,
    'rating': data['Rating'].values
})

---
Train Explicit Model

In [7]:
m1 = turicreate.recommender.ranking_factorization_recommender.create(sf, user_id='user', item_id='movie', target='rating', solver='ials')

Preparing data set.

Data has 100000 observations with 943 users and 1682 items.

Data prepared in: 0.143658s

Training ranking_factorization_recommender for recommendations.

+------------------------------+--------------------------------------------------+----------+

...

| 23      | 851.364ms    | 2.9767                    |

| 24      | 882.894ms    | 2.97668                   |

| FINAL   | 882.939ms    | 2.97668                   |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

In [34]:
user = 2
res = m1.recommend([user], k=10)
user_watched = m1.get_num_items_per_user()
user_watched = sf[sf['user']==user]['movie']
res['already_liked'] = np.in1d(res['movie'], user_watched).astype(str)
res['movie'] = movie_arr[res['movie']]
res.head()

user,movie,score,rank,already_liked
2,"Crossing Guard, The(1995) ...",0.8964557647705078,1,False
2,Dances with Wolves (1990),0.8265966176986694,2,False
2,Star Trek: The Wrath ofKhan (1982) ...,0.8018900752067566,3,False
2,"Remains of the Day, The(1993) ...",0.779320240020752,4,False
2,Tombstone (1993),0.7713500261306763,5,False
2,Mimic (1997),0.7664505243301392,6,False
2,"Substitute, The (1996)",0.7470420002937317,7,False
2,One Fine Day (1996),0.7338156700134277,8,False
2,Desperado (1995),0.7283619046211243,9,False
2,Man of the House (1995),0.6757052540779114,10,False


---
Train Implicit Model

In [36]:
sf = turicreate.SFrame({
    'user': data['UserId'].values,
    'movie': data['MovieId'].values,
})

In [37]:
m2 = turicreate.recommender.ranking_factorization_recommender.create(sf, user_id='user', item_id='movie', solver='ials')

Preparing data set.

Data has 100000 observations with 943 users and 1682 items.

Data prepared in: 0.077917s

Training ranking_factorization_recommender for recommendations.

+------------------------------+--------------------------------------------------+----------+

...

| 23      | 838.048ms    | 0.444652                  |

| 24      | 869.775ms    | 0.444641                  |

| FINAL   | 869.836ms    | 0.444641                  |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

In [38]:
user = 2
res = m2.recommend([user], k=10)
user_watched = m1.get_num_items_per_user()
user_watched = sf[sf['user']==user]['movie']
res['already_liked'] = np.in1d(res['movie'], user_watched).astype(str)
res['movie'] = movie_arr[res['movie']]
res.head()

user,movie,score,rank,already_liked
2,Dances with Wolves (1990),0.6902086734771729,1,False
2,Star Trek: The Wrath ofKhan (1982) ...,0.6851856708526611,2,False
2,"Remains of the Day, The(1993) ...",0.6771501898765564,3,False
2,"Crossing Guard, The(1995) ...",0.6578421592712402,4,False
2,"Substitute, The (1996)",0.6329699158668518,5,False
2,Desperado (1995),0.6253886818885803,6,False
2,Naked (1993),0.6092404723167419,7,False
2,One Fine Day (1996),0.6016614437103271,8,False
2,Only You (1994),0.5534247159957886,9,False
2,"Secret of Roan Inish, The(1994) ...",0.5499218106269836,10,False
